In [1]:
import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "disabled"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
!pip install --upgrade unsloth peft bitsandbytes accelerate trl

  Using cached trl-0.26.2-py3-none-any.whl.metadata (11 kB)


In [3]:
!pip install evaluate

# **IMPORTS**

In [4]:
from unsloth import FastLanguageModel
import torch
from torch.nn.utils.rnn import pad_sequence

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-12-19 12:24:31.946294: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766147071.968843     387 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766147071.976334     387 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment 

🦥 Unsloth Zoo will now patch everything to make training faster!


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using

In [5]:
import scipy
import json
import re
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
import evaluate
from scipy.stats import spearmanr, kendalltau
from datasets import Dataset
from tqdm import tqdm
import glob


from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling, EarlyStoppingCallback
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training, PeftModel

# **CARGO LOS DATOS**

In [6]:
#summ = []
#with open("/kaggle/input/summ-dataset/summ.jsonl", "r", encoding="utf-8") as file:
#    for line in file:
#        summ.append(json.loads(line))
#
#df = pd.DataFrame(summ)

In [7]:
def load_jsonl(path):
    data = []
    try:
        with open(path, "r", encoding="utf-8") as f:
            for line in f:
                data.append(json.loads(line))
        return pd.DataFrame(data)
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo {path}")
        return pd.DataFrame()

df_es = load_jsonl("/kaggle/input/basse-es-jsonl/BASSE_es.jsonl")
df_eu = load_jsonl("/kaggle/input/basse-eu-jsonl/BASSE_eu.jsonl")
df_test_gl = load_jsonl("/kaggle/input/basse-gl-jsonl/BASSE.gl.jsonl")

# **DIVIDO EL CONJUNTO DE DATOS**

In [11]:
# 1 -> Separamos test sets para ES y EU (10%)
train_dev_es, test_es = train_test_split(df_es, test_size=9, random_state=42, shuffle=True)
train_dev_eu, test_eu = train_test_split(df_eu, test_size=9, random_state=42, shuffle=True)

# 2 -> Creamos train sets (80% total) y dev sets (10% total) para ES y EU
train_es, dev_es = train_test_split(train_dev_es, test_size=8, random_state=42, shuffle=True)
train_eu, dev_eu = train_test_split(train_dev_eu, test_size=8, random_state=42, shuffle=True)

# 3 -> Shuffle ES + EU
df_train = pd.concat([train_es, train_eu]).sample(frac=1, random_state=42).reset_index(drop=True)
df_dev = pd.concat([dev_es, dev_eu]).sample(frac=1, random_state=42).reset_index(drop=True)

# 4 -> Ponemos nombres claros a los test
df_test_es = test_es.reset_index(drop=True)
df_test_eu = test_eu.reset_index(drop=True)
# df_test_gl 

print(f"TRAIN:    {len(df_train)} (ES: {len(train_es)} + EU: {len(train_eu)})")
print(f"DEV:      {len(df_dev)} (ES: {len(dev_es)} + EU: {len(dev_eu)})")
print("-" * 30)
print(f"TEST ES:  {len(df_test_es)}")
print(f"TEST EU:  {len(df_test_eu)}")
print(f"TEST GL:  {len(df_test_gl)}")

TRAIN:    56 (ES: 28 + EU: 28)
DEV:      16 (ES: 8 + EU: 8)
------------------------------
TEST ES:  9
TEST EU:  9
TEST GL:  15


# **COMO UN RESUMEN TIENE MAS DE UNA ANOTACIÓN, HAGO LA MEDIA**

In [12]:
def extraer_resumenes(df):
    res = []

    for _, row in df.iterrows():
        original = row.get('original_document', '')
        
        model_data = row.get('model_summaries', {})
        
        if isinstance(model_data, dict):
            for model_name, contenido in model_data.items():
                resumen = contenido.get("summ", None)
                anns = contenido.get("anns", {})
                
                coherence_vals = anns.get("Coherence", None)

                if resumen and coherence_vals:
                    if isinstance(coherence_vals, list):
                        score = np.mean(coherence_vals)
                    else:
                        score = float(coherence_vals)
                        
                    media = int(round(score))
                    
                    res.append((resumen, media, original))

    return pd.DataFrame(res, columns=["resumen", "coherence", "original_document"])

In [13]:
df_train_redondeo = extraer_resumenes(df_train)
df_dev_redondeo = extraer_resumenes(df_dev)
df_test_es_redondeo = extraer_resumenes(df_test_es)
df_test_eu_redondeo = extraer_resumenes(df_test_eu)
df_test_gl_redondeo = extraer_resumenes(df_test_gl)

In [14]:
print(f"Ejemplos procesados TRAIN: {len(df_train_redondeo)}")
print(f"Ejemplos procesados DEV: {len(df_dev_redondeo)}")
print(f"Ejemplos procesados TEST es: {len(df_test_es_redondeo)}")
print(f"Ejemplos procesados TEST eu: {len(df_test_eu_redondeo)}")
print(f"Ejemplos procesados TEST gl: {len(df_test_gl_redondeo)}")

Ejemplos procesados TRAIN: 1248
Ejemplos procesados DEV: 336
Ejemplos procesados TEST es: 198
Ejemplos procesados TEST eu: 198
Ejemplos procesados TEST gl: 150


# **CONSTRUYO EL PROMPT**

In [15]:
def construct_metric_prompt_simple(row, target_metric):
    """Construye un prompt de instrucción para evaluar un resumen usando una métrica específica"""
    
    summary = row['resumen']
    score = row[target_metric]
    
    prompt = f"""[INSTRUCCIÓN]
As an expert evaluator, analyze the following summary.
Evaluate solely the {target_metric} criterion on a scale of 1 to 5, where 1 is the lowest score and 5 is the highest.

Criterion: Coherence: the collective quality of all sentences. We align this dimension with the DUC quality question (Dang, 2005) of structure and coherence whereby 'the summary should be well-structured and well-organized. 
The summary should not just be a heap of related information, but should build from sentence to sentence to a coherent body of information about a topic.' 
To clarify, if you were a language teacher, how would you grade the flow of ideas? Does the summary contain connectors to conect ideas? It doesn't matter if there are grammatical errors or incorrect information. 
If the summary is simply a list of events, we penalize it. We do not penalize single sentences for not having connectors.

Score 1: The summary contains a bulletpoint list of events and there is no internal consistency (e.g., random words).
Score 2: The summary contains a bulletpoint list of events, but there is no consistency between the phrases in each bulletpoint (e.g., 1. Keyboard, 2. Watching TV).
Score 3: The summary contains a bulletpoint list of events, pero the phrases in each bullet point are developed well.
Score 4: The summary contains implicit coherence, but not explicit. That means that there are no temporal connectors or discourse markers, but the sentences have a temporal/causal flow.
Score 5: The summary contains explicit coherence via textual resources such as temporal connector or discourse markers that mark the relationship between ideas and sentences. 

Provide only the score for the criterion indicated below in the exact format. **Do not add any justification, explanation, or additional text**, just the score.

Expected output format:
- [COHERENCE]: score

[SUMMARY] This is the summary to evaluate:
{summary}

Provide your evaluation in the exact format: [COHERENCE]: (N) where N is a number from 1 to 5
"""
    
    response = f"[{target_metric.upper()}]: {int(round(score))}"
    return {"prompt": prompt, "completion": response}

In [16]:
# Función prepare_metric_dataset ahora puede usar los 3 test sets
def prepare_metric_datasets(df_train, df_dev, df_test_es, df_test_eu, df_test_gl, target_metric):
    def build_data(df, split_name):
        data = []
        for idx, row in tqdm(df.iterrows(), total=len(df), desc=f"Preparando {split_name} para {target_metric}"):
            data.append(construct_metric_prompt_simple(row, target_metric))
        return data

    train_data = build_data(df_train, "train")
    dev_data = build_data(df_dev, "dev")
    test_es_data = build_data(df_test_es, "test_es")
    test_eu_data = build_data(df_test_eu, "test_eu")
    test_gl_data = build_data(df_test_gl, "test_gl")


    train_dataset = Dataset.from_dict({
        "prompt": [d["prompt"] for d in train_data],
        "completion": [d["completion"] for d in train_data]
    })
    
    dev_dataset = Dataset.from_dict({
        "prompt": [d["prompt"] for d in dev_data],
        "completion": [d["completion"] for d in dev_data]
    })
    
    test_es_dataset = Dataset.from_dict({
        "prompt": [d["prompt"] for d in test_es_data],
        "completion": [d["completion"] for d in test_es_data]
    })

    test_eu_dataset = Dataset.from_dict({
        "prompt": [d["prompt"] for d in test_eu_data],
        "completion": [d["completion"] for d in test_eu_data]
    })

    test_gl_dataset = Dataset.from_dict({
        "prompt": [d["prompt"] for d in test_gl_data],
        "completion": [d["completion"] for d in test_gl_data]
    })

    return train_dataset, dev_dataset, test_es_dataset, test_eu_dataset, test_gl_dataset


In [17]:
target_metric = "coherence"
#train_dataset, dev_dataset, test_dataset = prepare_metric_datasets(df_train_redondeo, df_dev_redondeo, df_test_redondeo, target_metric)
train_dataset, dev_dataset, test_es_dataset, test_eu_dataset, test_gl_dataset = prepare_metric_datasets(df_train_redondeo, df_dev_redondeo, df_test_es_redondeo, df_test_eu_redondeo, df_test_gl_redondeo, target_metric)

print(f"\nDataset de entrenamiento: {len(train_dataset)} ejemplos")
print(f"Dataset de desarrollo: {len(dev_dataset)} ejemplos")
print(f"Dataset de prueba es: {len(test_es_dataset)} ejemplos")
print(f"Dataset de prueba eu: {len(test_eu_dataset)} ejemplos")
print(f"Dataset de prueba gl: {len(test_gl_dataset)} ejemplos")

Preparando test_gl para coherence: 100%|██████████| 150/150 [00:00<00:00, 23224.27it/s]


Dataset de entrenamiento: 1248 ejemplos
Dataset de desarrollo: 336 ejemplos
Dataset de prueba es: 198 ejemplos
Dataset de prueba eu: 198 ejemplos
Dataset de prueba gl: 150 ejemplos


# **CARGO EL MODELO**

In [18]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-bnb-4bit", 
    max_seq_length = 5000,
    dtype = None,
    load_in_4bit = True,  
)
FastLanguageModel.for_training(model)

==((====))==  Unsloth 2025.12.7: Fast Mistral patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layer

In [19]:
#from kaggle_secrets import UserSecretsClient
#user_secrets = UserSecretsClient()
#secret_value_0 = user_secrets.get_secret("wandb")

In [20]:
#wandb.login(key=secret_value_0)
#run = wandb.init(
#    project='Coherence_TFG_Leire_SinDataAumentation',
#    name="run_lora_r8_bs4_alpha16_balanceado",
#    job_type="training",
#    anonymous="allow",
#    config={
#        "batch_size": 4,
#        "gradient_accumulation_steps": 2,
#        "learning_rate": 5e-5,
#        "lora_r": 8,
#        "lora_alpha": 16,
#        "dropout": 0.1,
#        "epochs": 2,
#    },
#    reinit=True
#)

In [21]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 8, 
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0.1,
    bias = "none",    
    use_gradient_checkpointing = "unsloth", 
    random_state = 3407,
    use_rslora = False,  
    loftq_config = None, 
)


Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.1.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.12.7 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [22]:
model.print_trainable_parameters()

trainable params: 20,971,520 || all params: 7,262,703,616 || trainable%: 0.2888


In [23]:
def preprocess(example):
    prompt = example["prompt"].strip()
    completion = example["completion"].strip()

    full_text = f"{prompt}\n### Completion:\n{completion}"
    input_text = f"{prompt}\n### Completion:\n"

    tokenized = tokenizer(full_text, return_tensors="pt", padding=False, truncation=True)
    input_ids = tokenized["input_ids"][0]
    attention_mask = tokenized["attention_mask"][0]

    labels = input_ids.clone()
    prompt_len = len(tokenizer(input_text, return_tensors="pt").input_ids[0])
    labels[:prompt_len] = -100 

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels,
    }


In [24]:
train_dataset = train_dataset.map(preprocess)
dev_dataset = dev_dataset.map(preprocess)

Map:   0%|          | 0/1248 [00:00<?, ? examples/s]

Map:   0%|          | 0/336 [00:00<?, ? examples/s]

In [25]:
class DataCollator:
    def __call__(self, features):
        # Asegurar que todos son tensores
        input_ids = [torch.tensor(f["input_ids"], dtype=torch.long) for f in features]
        attention_mask = [torch.tensor(f["attention_mask"], dtype=torch.long) for f in features]
        labels = [torch.tensor(f["labels"], dtype=torch.long) for f in features]

        input_ids = pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
        attention_mask = pad_sequence(attention_mask, batch_first=True, padding_value=0)
        labels = pad_sequence(labels, batch_first=True, padding_value=-100)

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels,
        }

In [26]:
training_args = TrainingArguments(
    output_dir="outputs",
    per_device_train_batch_size=2, 
    gradient_accumulation_steps=4, 
    gradient_checkpointing=True, # I added this. O meu input é longo e non quero erro OOM
    learning_rate=5e-5,
    num_train_epochs=3,
    fp16=True,
    bf16=False,
    logging_steps=1,
    eval_strategy="steps",    
    eval_steps=20,
    save_strategy="steps",          
    save_steps=20,
    save_total_limit=1,             
    load_best_model_at_end=True,   
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    report_to=None,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    data_collator=DataCollator(),
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)],
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


# **ENTRENO EL MODELO**

In [27]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,248 | Num Epochs = 3 | Total steps = 468
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 20,971,520 of 7,262,703,616 (0.29% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
20,0.158000,0.184427
40,0.227100,0.172569
60,0.223100,0.220811
80,0.117500,0.163860
100,0.123200,0.167740
120,0.113000,0.163179
140,0.117900,0.169440
160,0.107000,0.162242
180,0.093700,0.168990
200,0.110000,0.166295


Unsloth: Not an error, but MistralForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


# **GUARDO EL MODELO ENTRENADO**

In [28]:
trainer.save_model("./output/final_model")
tokenizer.save_pretrained("./output/final_model")

('./output/final_model/tokenizer_config.json',
 './output/final_model/special_tokens_map.json',
 './output/final_model/tokenizer.model',
 './output/final_model/added_tokens.json',
 './output/final_model/tokenizer.json')

In [29]:
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name =  "./output/final_model",
    max_seq_length = 5000,
    load_in_4bit = True,
)

==((====))==  Unsloth 2025.12.7: Fast Mistral patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: Will load ./output/final_model as a legacy tokenizer.


# **LAS REFERENCIAS**

In [30]:
def get_references(eval_dataset):
    respuestas = []
    for x in eval_dataset:
        respuesta = [x["response"]]
        match = re.search(r"\[COHERENCE\]:\s*(\d+)", respuesta[0])
        if match:
            respuestas.append(int(match.group(1)))
        else: 
            respuestas.append(None)

    return respuestas

In [31]:
test_eval_dataset_es = []
for i in range(len(test_es_dataset)):
    test_eval_dataset_es.append({
        "prompt": test_es_dataset[i]["prompt"],
        "response": test_es_dataset[i]["completion"]
    })

In [32]:
referencias_es = get_references(test_eval_dataset_es)
print("Referencias ES:", referencias_es)

Referencias ES: [3, 3, 3, 3, 4, 4, 3, 5, 4, 4, 3, 4, 4, 4, 3, 4, 4, 3, 3, 4, 3, 3, 2, 2, 2, 3, 5, 3, 4, 4, 5, 3, 4, 4, 4, 3, 4, 3, 2, 3, 5, 4, 5, 4, 2, 2, 5, 5, 5, 5, 5, 5, 3, 5, 5, 5, 3, 5, 4, 4, 3, 5, 4, 4, 3, 3, 3, 5, 4, 3, 5, 5, 3, 3, 5, 5, 5, 3, 4, 5, 3, 3, 4, 5, 4, 3, 3, 3, 4, 4, 3, 4, 5, 5, 3, 4, 4, 4, 3, 4, 5, 3, 3, 4, 5, 3, 5, 2, 3, 4, 5, 3, 5, 5, 5, 3, 5, 5, 5, 3, 4, 4, 3, 3, 5, 5, 5, 5, 5, 3, 4, 3, 3, 4, 5, 3, 5, 5, 5, 3, 4, 5, 4, 3, 4, 5, 5, 3, 4, 4, 5, 5, 5, 2, 4, 2, 3, 5, 4, 3, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 2, 4, 4, 5, 5, 5, 4, 3, 3, 2, 4, 5, 3, 4, 3, 3, 3, 4, 4, 5, 3, 4, 5, 5, 3, 5, 5]


In [33]:
test_eval_dataset_eu = []
for i in range(len(test_eu_dataset)):
    test_eval_dataset_eu.append({
        "prompt": test_eu_dataset[i]["prompt"],
        "response": test_eu_dataset[i]["completion"]
    })
    
referencias_eu = get_references(test_eval_dataset_eu)
print("Referencias EU:", referencias_eu)

Referencias EU: [2, 3, 2, 2, 4, 2, 3, 3, 5, 4, 3, 4, 3, 3, 3, 4, 4, 3, 3, 4, 4, 1, 3, 1, 2, 2, 3, 2, 5, 5, 3, 3, 3, 4, 3, 3, 3, 1, 4, 3, 5, 5, 3, 4, 1, 1, 4, 4, 2, 2, 4, 3, 2, 3, 3, 3, 2, 4, 3, 3, 1, 2, 3, 3, 3, 3, 3, 3, 2, 3, 2, 5, 5, 3, 3, 4, 3, 3, 4, 4, 3, 3, 4, 3, 3, 4, 3, 3, 5, 1, 2, 4, 5, 5, 3, 5, 4, 4, 3, 4, 2, 4, 3, 4, 4, 3, 3, 3, 3, 4, 2, 3, 4, 4, 4, 3, 4, 5, 3, 3, 5, 4, 3, 3, 4, 4, 5, 4, 4, 4, 5, 3, 2, 4, 4, 3, 5, 5, 4, 3, 4, 4, 5, 3, 4, 4, 4, 3, 4, 4, 5, 4, 5, 3, 4, 3, 3, 3, 3, 2, 4, 4, 4, 3, 5, 4, 4, 3, 4, 4, 4, 3, 4, 4, 5, 5, 4, 3, 5, 3, 5, 4, 4, 3, 4, 4, 4, 3, 4, 4, 4, 3, 4, 5, 5, 3, 5, 5]


In [34]:
test_eval_dataset_gl = []
for i in range(len(test_gl_dataset)):
    test_eval_dataset_gl.append({
        "prompt": test_gl_dataset[i]["prompt"],
        "response": test_gl_dataset[i]["completion"]
    })
    
referencias_gl = get_references(test_eval_dataset_gl)
print("Referencias GL:", referencias_gl)

Referencias GL: [4, 3, 5, 4, 3, 2, 4, 3, 4, 3, 4, 3, 4, 3, 2, 4, 3, 3, 4, 4, 4, 3, 4, 3, 2, 5, 4, 3, 5, 4, 4, 3, 4, 4, 3, 5, 4, 2, 4, 5, 3, 3, 4, 5, 3, 5, 5, 4, 5, 5, 3, 3, 3, 4, 4, 4, 4, 2, 5, 4, 4, 2, 4, 5, 3, 5, 4, 3, 4, 4, 4, 3, 4, 3, 3, 4, 5, 3, 5, 4, 4, 3, 4, 4, 3, 5, 1, 2, 5, 5, 4, 3, 4, 4, 3, 4, 4, 3, 4, 5, 5, 3, 4, 5, 2, 4, 4, 3, 4, 5, 4, 3, 4, 4, 2, 4, 5, 3, 4, 4, 4, 3, 4, 4, 2, 4, 4, 3, 5, 5, 5, 3, 5, 4, 3, 4, 4, 2, 5, 5, 4, 2, 4, 5, 3, 4, 5, 2, 5, 5]


# **EVALUO EL MODELO**

In [35]:
def evaluate_metric_model(model, dataset, tokenizer, metric):
    all_predictions = []
    model.eval()
    all_predictions = []
    all_references = []

    
    # Iterate over the dataset
    for idx, example in enumerate(tqdm(dataset, desc=f"Evaluating {metric} model")):

        prompt = example["prompt"] 
        
        inputs = tokenizer(
            prompt,
            return_tensors="pt",
            truncation=True,
            max_length=5000,
            padding=True
        )
        inputs = inputs.to(model.device)
        with torch.no_grad():
            outputs = model.generate(
                input_ids=inputs["input_ids"],
                attention_mask=inputs["attention_mask"],
                max_new_tokens=50,
                do_sample=False,  # Generación determinista (greedy)
                pad_token_id=tokenizer.eos_token_id
            )
        
        prompt_length = len(inputs["input_ids"][0])
        new_tokens = outputs[0][prompt_length:]
        generated_text = tokenizer.decode(new_tokens, skip_special_tokens=True)
        match = re.search(r"\[COHERENCE\]:\s*(\d+)", generated_text)
        print(generated_text)
        if match:
            all_predictions.append(float(match.group(1)))
        else:
            all_predictions.append(None)

    
    return all_predictions



# **TEST ES**

In [36]:
predictions_es = evaluate_metric_model(model, test_es_dataset,  tokenizer, "Coherence")

Evaluating Coherence model:   1%|          | 1/198 [00:01<06:20,  1.93s/it]

[COHERENCE]: 4


Evaluating Coherence model:   1%|          | 2/198 [00:03<05:24,  1.66s/it]

[COHERENCE]: 4


Evaluating Coherence model:   2%|▏         | 3/198 [00:04<05:05,  1.56s/it]

...

[COHERENCE]: 3


Evaluating Coherence model:  99%|█████████▉| 197/198 [05:37<00:01,  1.44s/it]

[COHERENCE]: 4


Evaluating Coherence model: 100%|██████████| 198/198 [05:41<00:00,  1.73s/it]

[COHERENCE]: 5

[SUMMARY] This is the summary to evaluate:
El Festival de Cine Judío de Barcelona ofrece 15 filmes en la Filmoteca

Provide your


In [37]:
print("ES test:", predictions_es)

ES test: [4.0, 4.0, 3.0, 3.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 5.0, 3.0, 3.0, 4.0, 5.0, 4.0, 3.0, 3.0, 3.0, 4.0, 4.0, 3.0, 5.0, 5.0, 4.0, 3.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 3.0, 3.0, 4.0, 4.0, 4.0, 4.0, 3.0, 3.0, 5.0, 4.0, 3.0, 4.0, 5.0, 5.0, 3.0, 5.0, 5.0, 5.0, 4.0, 5.0, 4.0, 4.0, 3.0, 5.0, 4.0, 4.0, 3.0, 3.0, 3.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 3.0, 3.0, 4.0, 4.0, 4.0, 4.0, 3.0, 3.0, 4.0, 4.0, 3.0, 5.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 3.0, 3.0, 4.0, 4.0, 3.0, 4.0, 3.0, 3.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 3.0, 3.0, 4.0, 5.0, 4.0, 5.0, 4.0, 3.0, 4.0, 3.0, 3.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 5.0, 3.0, 4.0, 5.0, 5.0, 3.0, 4.0, 4.0, 4.0, 5.0, 4.0, 3.0, 4.0, 3.0, 3.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 4.0, 4.0, 3.0, 3.0, 3.0, 3.0, 5.0, 5.0, 3.0, 5.0, 3.0, 4.0, 3.0, 4.0, 5.0, 5.0, 3.0, 5.0, 4.0, 4.0, 3.0, 4.0, 5.0]


In [38]:
cantidad_none = predictions_es.count(None)
total_es = len(predictions_es)
porcentaje_es = cantidad_none / total_es

print(f"Cantidad de valores None: {cantidad_none}, Total de predicciones: {total_es}")
print(f"Porcentaje de Nones: {porcentaje_es:.2%}")

Cantidad de valores None: 0, Total de predicciones: 198
Porcentaje de Nones: 0.00%


## **QUITO LOS NONE**

In [39]:
filtrados = [(h, p) for h, p in zip(referencias_es, predictions_es) if h is not None and p is not None]
hum_limpio, predictions_limpio = zip(*filtrados)

In [40]:
print(hum_limpio)
print(predictions_limpio)

(3, 3, 3, 3, 4, 4, 3, 5, 4, 4, 3, 4, 4, 4, 3, 4, 4, 3, 3, 4, 3, 3, 2, 2, 2, 3, 5, 3, 4, 4, 5, 3, 4, 4, 4, 3, 4, 3, 2, 3, 5, 4, 5, 4, 2, 2, 5, 5, 5, 5, 5, 5, 3, 5, 5, 5, 3, 5, 4, 4, 3, 5, 4, 4, 3, 3, 3, 5, 4, 3, 5, 5, 3, 3, 5, 5, 5, 3, 4, 5, 3, 3, 4, 5, 4, 3, 3, 3, 4, 4, 3, 4, 5, 5, 3, 4, 4, 4, 3, 4, 5, 3, 3, 4, 5, 3, 5, 2, 3, 4, 5, 3, 5, 5, 5, 3, 5, 5, 5, 3, 4, 4, 3, 3, 5, 5, 5, 5, 5, 3, 4, 3, 3, 4, 5, 3, 5, 5, 5, 3, 4, 5, 4, 3, 4, 5, 5, 3, 4, 4, 5, 5, 5, 2, 4, 2, 3, 5, 4, 3, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 2, 4, 4, 5, 5, 5, 4, 3, 3, 2, 4, 5, 3, 4, 3, 3, 3, 4, 4, 5, 3, 4, 5, 5, 3, 5, 5)
(4.0, 4.0, 3.0, 3.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 5.0, 3.0, 3.0, 4.0, 5.0, 4.0, 3.0, 3.0, 3.0, 4.0, 4.0, 3.0, 5.0, 5.0, 4.0, 3.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 3.0, 3.0, 4.0, 4.0, 4.0, 4.0, 3.0, 3.0, 5.0, 4.0, 3.0, 4.0, 5.0, 5.0, 3.0, 5.0, 5.0, 5.0, 4.0, 5.0, 4.0, 4.0, 3.0, 5.0, 4.0, 4.0, 3.0, 3.0, 3.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 3.0,

## **MÉTRICAS**

## **SPEARMANR**

In [41]:
# Mide si el modelo puede ordenar los modelos de peor a mejor 
# en comparación con las puntuaciones humanas (no exactamente la misma puntuación)
s, p = spearmanr(hum_limpio, predictions_limpio)
print(f"Spearman ES: {s:.3f}, p-value: {p:.3f}")

Spearman ES: 0.725, p-value: 0.000


## **KENDALLTAU**

In [42]:
# Mide la capacidad para ordenar los resumenes según su calidad, 
# pero se calcula por parejas (de estos dos cuál es el mejor?)
tau, p_value = scipy.stats.kendalltau(hum_limpio, predictions_limpio)
print(f"Kendalltau ES: {tau:.3f}, p-value: {p_value:.3f}")

Kendalltau ES: 0.658, p-value: 0.000


## **MAE**

In [43]:
# Error medio absoluto
# Mide la distancia media entre las puntuaciones predichas por los modelos 
# y las dadas por los humanos
# Es decir, menos de 0,5 (ej.: humano -> 5, modelo -> 4.60)
mae = np.mean(np.abs(np.array(hum_limpio) - np.array(predictions_limpio)))
print(f"MAE ES: {mae}")

MAE ES: 0.4090909090909091


# **TEST EU**

In [44]:
predictions_eu = evaluate_metric_model(model, test_eu_dataset,  tokenizer, "Coherence")

Evaluating Coherence model:   1%|          | 1/198 [00:01<05:27,  1.66s/it]

[COHERENCE]: 3


Evaluating Coherence model:   1%|          | 2/198 [00:03<05:20,  1.63s/it]

[COHERENCE]: 4


Evaluating Coherence model:   2%|▏         | 3/198 [00:04<05:19,  1.64s/it]

...

[COHERENCE]: 3


Evaluating Coherence model:  99%|█████████▉| 197/198 [05:20<00:01,  1.38s/it]

[COHERENCE]: 4


Evaluating Coherence model: 100%|██████████| 198/198 [05:21<00:00,  1.62s/it]

[COHERENCE]: 4


In [45]:
print("EU test:", predictions_eu)

EU test: [3.0, 4.0, 3.0, 3.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 3.0, 3.0, 3.0, 3.0, 4.0, 3.0, 3.0, 3.0, 4.0, 3.0, 3.0, 4.0, 4.0, 3.0, 3.0, 4.0, 4.0, 3.0, 3.0, 4.0, 4.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 4.0, 3.0, 3.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 3.0, 3.0, 3.0, 3.0, 3.0, 4.0, 3.0, 3.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 3.0, 4.0, 3.0, 3.0, 5.0, 3.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 3.0, 3.0, 3.0, 3.0, 3.0, 4.0, 4.0, 3.0, 4.0, 3.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 4.0, 4.0, 3.0, 4.0, 3.0, 3.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 4.0, 4.0, 3.0, 4.0, 3.0, 3.0, 4.0, 3.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 4.0, 4.0, 3.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0]


In [46]:
cantidad_none_eu = predictions_eu.count(None)
total_eu = len(predictions_eu)
porcentaje_eu = cantidad_none_eu / total_eu

print(f"Cantidad de valores None: {cantidad_none_eu}, Total de predicciones: {total_eu}")
print(f"Porcentaje de Nones: {porcentaje_eu:.2%}")

Cantidad de valores None: 0, Total de predicciones: 198
Porcentaje de Nones: 0.00%


## **QUITO LOS NONE**

In [47]:
filtrados_eu = [(h, p) for h, p in zip(referencias_eu, predictions_eu) if h is not None and p is not None]
hum_limpio_eu, predictions_limpio_eu = zip(*filtrados_eu)

print(hum_limpio_eu)
print(predictions_limpio_eu)

(2, 3, 2, 2, 4, 2, 3, 3, 5, 4, 3, 4, 3, 3, 3, 4, 4, 3, 3, 4, 4, 1, 3, 1, 2, 2, 3, 2, 5, 5, 3, 3, 3, 4, 3, 3, 3, 1, 4, 3, 5, 5, 3, 4, 1, 1, 4, 4, 2, 2, 4, 3, 2, 3, 3, 3, 2, 4, 3, 3, 1, 2, 3, 3, 3, 3, 3, 3, 2, 3, 2, 5, 5, 3, 3, 4, 3, 3, 4, 4, 3, 3, 4, 3, 3, 4, 3, 3, 5, 1, 2, 4, 5, 5, 3, 5, 4, 4, 3, 4, 2, 4, 3, 4, 4, 3, 3, 3, 3, 4, 2, 3, 4, 4, 4, 3, 4, 5, 3, 3, 5, 4, 3, 3, 4, 4, 5, 4, 4, 4, 5, 3, 2, 4, 4, 3, 5, 5, 4, 3, 4, 4, 5, 3, 4, 4, 4, 3, 4, 4, 5, 4, 5, 3, 4, 3, 3, 3, 3, 2, 4, 4, 4, 3, 5, 4, 4, 3, 4, 4, 4, 3, 4, 4, 5, 5, 4, 3, 5, 3, 5, 4, 4, 3, 4, 4, 4, 3, 4, 4, 4, 3, 4, 5, 5, 3, 5, 5)
(3.0, 4.0, 3.0, 3.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 3.0, 3.0, 3.0, 3.0, 4.0, 3.0, 3.0, 3.0, 4.0, 3.0, 3.0, 4.0, 4.0, 3.0, 3.0, 4.0, 4.0, 3.0, 3.0, 4.0, 4.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 4.0, 3.0, 3.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 3.0, 3.0, 3.0, 3.0, 3.0, 4.0, 3.0, 3.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0,

## **MÉTRICAS**

## **SPEARMANR**

In [48]:
# Mide si el modelo puede ordenar los modelos de peor a mejor 
# en comparación con las puntuaciones humanas (no exactamente la misma puntuación)
s, p = spearmanr(hum_limpio_eu, predictions_limpio_eu)
print(f"Spearman EU: {s:.3f}, p-value: {p:.3f}")

Spearman EU: 0.613, p-value: 0.000


## **KENDALLTAU**

In [49]:
# Mide la capacidad para ordenar los resumenes según su calidad, 
# pero se calcula por parejas (de estos dos cuál es el mejor?)
tau, p_value = scipy.stats.kendalltau(hum_limpio_eu, predictions_limpio_eu)
print(f"Kendalltau EU: {tau:.3f}, p-value: {p_value:.3f}")

Kendalltau EU: 0.565, p-value: 0.000


## **MAE**

In [50]:
# Error medio absoluto
# Mide la distancia media entre las puntuaciones predichas por los modelos 
# y las dadas por los humanos
# Es decir, menos de 0,5 (ej.: humano -> 5, modelo -> 4.60)
mae = np.mean(np.abs(np.array(hum_limpio_eu) - np.array(predictions_limpio_eu)))
print(f"MAE EU: {mae}")

MAE EU: 0.4898989898989899


# **TEST GL**

In [51]:
predictions_gl = evaluate_metric_model(model, test_gl_dataset,  tokenizer, "Coherence")
print("GL test:", predictions_gl)

Evaluating Coherence model:   1%|          | 1/150 [00:01<03:36,  1.45s/it]

[COHERENCE]: 4


Evaluating Coherence model:   1%|▏         | 2/150 [00:02<03:42,  1.50s/it]

[COHERENCE]: 3


Evaluating Coherence model:   2%|▏         | 3/150 [00:04<03:37,  1.48s/it]

...

[COHERENCE]: 3


Evaluating Coherence model:  99%|█████████▉| 149/150 [03:45<00:01,  1.34s/it]

[COHERENCE]: 4


Evaluating Coherence model: 100%|██████████| 150/150 [03:47<00:00,  1.51s/it]

[COHERENCE]: 4
GL test: [4.0, 3.0, 4.0, 4.0, 3.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 3.0, 3.0, 4.0, 4.0, 3.0, 4.0, 4.0, 5.0, 3.0, 5.0, 4.0, 3.0, 5.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 5.0, 4.0, 4.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 5.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 3.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 4.0, 3.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 3.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 3.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 4.0, 5.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0]


In [52]:
cantidad_none_gl = predictions_gl.count(None)
total_gl = len(predictions_gl)
porcentaje_gl = cantidad_none_gl / total_gl

print(f"Cantidad de valores None: {cantidad_none_gl}, Total de predicciones: {total_gl}")
print(f"Porcentaje de Nones: {porcentaje_gl:.2%}")

Cantidad de valores None: 0, Total de predicciones: 150
Porcentaje de Nones: 0.00%


## **QUITO LOS NONE**

In [53]:
filtrados_gl = [(h, p) for h, p in zip(referencias_gl, predictions_gl) if h is not None and p is not None]
hum_limpio_gl, predictions_limpio_gl = zip(*filtrados_gl)

In [54]:
print(hum_limpio_gl)
print(predictions_limpio_gl)

(4, 3, 5, 4, 3, 2, 4, 3, 4, 3, 4, 3, 4, 3, 2, 4, 3, 3, 4, 4, 4, 3, 4, 3, 2, 5, 4, 3, 5, 4, 4, 3, 4, 4, 3, 5, 4, 2, 4, 5, 3, 3, 4, 5, 3, 5, 5, 4, 5, 5, 3, 3, 3, 4, 4, 4, 4, 2, 5, 4, 4, 2, 4, 5, 3, 5, 4, 3, 4, 4, 4, 3, 4, 3, 3, 4, 5, 3, 5, 4, 4, 3, 4, 4, 3, 5, 1, 2, 5, 5, 4, 3, 4, 4, 3, 4, 4, 3, 4, 5, 5, 3, 4, 5, 2, 4, 4, 3, 4, 5, 4, 3, 4, 4, 2, 4, 5, 3, 4, 4, 4, 3, 4, 4, 2, 4, 4, 3, 5, 5, 5, 3, 5, 4, 3, 4, 4, 2, 5, 5, 4, 2, 4, 5, 3, 4, 5, 2, 5, 5)
(4.0, 3.0, 4.0, 4.0, 3.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 3.0, 3.0, 4.0, 4.0, 3.0, 4.0, 4.0, 5.0, 3.0, 5.0, 4.0, 3.0, 5.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 5.0, 4.0, 4.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 5.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 3.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 4.0, 3.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 3.0, 4.0, 4.0, 3.0, 4.0, 4.0

## **MÉTRICAS**

## **SPEARMANR**

In [55]:
# Mide si el modelo puede ordenar los modelos de peor a mejor 
# en comparación con las puntuaciones humanas (no exactamente la misma puntuación)
s, p = spearmanr(hum_limpio_gl, predictions_limpio_gl)
print(f"Spearman GL: {s:.3f}, p-value: {p:.3f}")

Spearman GL: 0.692, p-value: 0.000


## **KENDALLTAU**

In [56]:
# Mide la capacidad para ordenar los resumenes según su calidad, 
# pero se calcula por parejas (de estos dos cuál es el mejor?)
tau, p_value = scipy.stats.kendalltau(hum_limpio_gl, predictions_limpio_gl)
print(f"Kendalltau GL: {tau:.3f}, p-value: {p_value:.3f}")

Kendalltau GL: 0.635, p-value: 0.000


## **MAE**

In [57]:
# Error medio absoluto
# Mide la distancia media entre las puntuaciones predichas por los modelos 
# y las dadas por los humanos
# Es decir, menos de 0,5 (ej.: humano -> 5, modelo -> 4.60)
mae = np.mean(np.abs(np.array(hum_limpio_gl) - np.array(predictions_limpio_gl)))
print(f"MAE GL: {mae}")

MAE GL: 0.4266666666666667
